# Purpose
Generate a visual of the counties that have been visited.

# Setup
## Import - Packages

In [1]:
# -------------------------------------------------------------------------- #
# GENERAL #
import os
import sys

# -------------------------------------------------------------------------- #
# ANALYSIS
import numpy as np
import pandas as pd
import siuba as s

# -------------------------------------------------------------------------- #
# LOCAL #
from tracking_counties.config import SEED, ROOT_DIR

In [2]:
SEED

5590

In [3]:
ROOT_DIR

PosixPath('/Users/evancanfield/Projects/tracking_counties')